<a href="https://colab.research.google.com/github/Jakhongir0103/Pytorch_course_for_DL/blob/main/Exercises/data_loader_by_hand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
from torchvision.datasets import MNIST
from sklearn.metrics import accuracy_score
import numpy as np
import torch.nn as nn
import torch

# get as dataloaders
train_data = MNIST(root='mnist_data/train', train=True, download=True)
test_data = MNIST(root='mnist_data/test', train=False, download=True)

# convert to numpy
train_data_X, train_data_y = train_data.data.detach().numpy(), train_data.targets.detach().numpy()
test_data_X, test_data_y = test_data.data.detach().numpy(), test_data.targets.detach().numpy()

# # normalize
# train_data_X = train_data_X / 255
# test_data_X = test_data_X / 255

# print shapes
train_data_X.shape, train_data_y.shape, test_data_X.shape, test_data_y.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Build the model

In [2]:
class CNNClassifier(nn.Module):
    def __init__(self, in_shape=1, out_shape=10, hidden_unit=10):
        super().__init__()
        self.convolution_block = nn.Sequential(
            nn.Conv2d(in_channels=in_shape, out_channels=hidden_unit, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_unit, out_channels=hidden_unit, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=hidden_unit, out_channels=hidden_unit, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_unit, out_channels=hidden_unit, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classifier_block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=7*7*hidden_unit, out_features=out_shape)
        )
    def forward(self, x):
        x = self.convolution_block(x)
        x = self.classifier_block(x)
        return x

# Without data loaders

### Prepare the data

In [ ]:
# conver the train data to tensor
train_data_X_torch = torch.from_numpy(train_data_X).to(torch.float32).unsqueeze(dim=1)
train_data_y_torch = torch.from_numpy(train_data_y).to(torch.int64)

# conver the test data to tensor
test_data_X_torch = torch.from_numpy(test_data_X).to(torch.float32).unsqueeze(dim=1)
test_data_y_torch = torch.from_numpy(test_data_y).to(torch.int64)

# check the shape
train_data_X_torch.shape, train_data_y_torch.shape # Batch x Dimension(3 for RGB) x height x width

(torch.Size([60000, 1, 28, 28]), torch.Size([60000]))

### Train

In [ ]:
from time import time

# define the model
clf = CNNClassifier().to(device)

# loss and optimization
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=clf.parameters(), lr=0.01)

start = time()
losses = []

# train loop
for epoch in range(5):
  # train
  clf.train()

  for X, y in zip(train_data_X_torch, train_data_y_torch):
    # prepare the data
    X, y = X.to(device).unsqueeze(0), y.to(device).unsqueeze(0)

    # predict
    y_logit = clf(X)

    # loss
    loss = loss_fn(y_logit, y)
    losses.append(loss.cpu().detach().numpy())

    # update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # track loss
  print(f"Epoch {epoch}: loss - {np.mean(losses)}")

print("Time",time()-start)

Epoch 0: loss - 2.377725601196289
Epoch 1: loss - 2.377725601196289
Epoch 2: loss - 2.377725601196289
Epoch 3: loss - 2.3777241706848145
Epoch 4: loss - 2.3777239322662354
Time 631.7624289989471


In [ ]:
with torch.inference_mode():
  # predictions
  prediction = []

  # test
  clf.eval()
  for X, y in zip(test_data_X_torch, test_data_y_torch):
    X, y = X.to(device).unsqueeze(0), y.to(device).unsqueeze(0)

    # predict
    y_logit = clf(X)
    prediction.append(torch.argmax(y_logit, dim=1).item())

# get accuracy
acc = accuracy_score(test_data_y, prediction)
print(f"Mean accuracy: {np.mean(acc)}")

Mean accuracy: 0.0958


# With data loaders

### Prepare the data

In [51]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

class CustomDataLoader(Dataset):
  def __init__(self, X, y, transform=None, target_transform=None) -> None:
    self.data = X
    self.target = y
    self.transform = transform
    self.target_transform = target_transform

  def __getitem__(self, idx) -> None:
    return self.data[idx], self.target[idx]

  def __len__(self):
    return len(self.target)

train_dataset = CustomDataLoader(train_data_X, train_data_y, transforms.ToTensor())
test_dataset = CustomDataLoader(test_data_X, test_data_y, transforms.ToTensor())

### Train - batch size of 1

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=1)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1)

# define the model
clf = CNNClassifier().to(device)

# loss and optimization
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=clf.parameters(), lr=0.01)

start = time()
losses = []

# train loop
for epoch in range(5):
  # train
  clf.train()
  # iterate over the dataset
  for X, y in train_dataloader:
    # prepare the data
    X = X.to(device).to(torch.float32).unsqueeze(dim=1)
    y = y.to(device).to(torch.int64)

    # predict
    y_logit = clf(X)

    # loss
    loss = loss_fn(y_logit, y)
    losses.append(loss.cpu().detach().numpy())

    # update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # track loss
  print(f"Epoch {epoch}: loss - {np.mean(losses)}")

print("Time",time()-start)

Epoch 0: loss - 2.377725124359131
Epoch 1: loss - 2.377725124359131
Epoch 2: loss - 2.377725124359131
Epoch 3: loss - 2.37772536277771
Epoch 4: loss - 2.377725601196289
Time 692.4034841060638


In [ ]:
# evaluate - Batch size of 1 only
with torch.inference_mode():
  # predictions
  prediction = []
  # test
  clf.eval()
  for X, y in test_dataloader:
    # prepare the data
    X = X.to(device).to(torch.float32).unsqueeze(dim=1)
    y = y.to(device).to(torch.int64)
    # predict
    y_logit = clf(X)
    prediction.append(torch.argmax(y_logit, dim=1).item())

# get accuracy
acc = accuracy_score(test_data_y, prediction)
print(f"Mean accuracy: {np.mean(acc)}")

Mean accuracy: 0.0958


### Train - batch size of 32

In [52]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=32)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=32)

# define the model
clf = CNNClassifier().to(device)

# loss and optimization
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=clf.parameters(), lr=0.01)

start = time()
losses = []

# train loop
for epoch in range(5):
  # train
  clf.train()
  # iterate over the dataset
  for X, y in train_dataloader:
    # prepare the data
    X = X.to(device).to(torch.float32).unsqueeze(dim=1)
    y = y.to(device).to(torch.int64)

    # predict
    y_logit = clf(X)

    # loss
    loss = loss_fn(y_logit, y)
    losses.append(loss.cpu().detach().numpy())

    # update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # track loss
  print(f"Epoch {epoch}: loss - {np.mean(losses)}")

print("Time",time()-start)

Epoch 0: loss - 0.22659756243228912
Epoch 1: loss - 0.20030319690704346
Epoch 2: loss - 0.1883440911769867
Epoch 3: loss - 0.1798975169658661
Epoch 4: loss - 0.17217399179935455
Time 26.520280838012695


In [54]:
# evaluate - Batch size of 32
with torch.inference_mode():
  # accuracy
  accuracy = []

  # test
  clf.eval()
  for X, y in test_dataloader:
    # prepare the data
    X = X.to(device).to(torch.float32).unsqueeze(dim=1)
    y = y.to(device).to(torch.int64)

    # predict
    y_logit = clf(X)
    prediction = torch.argmax(y_logit, dim=1).tolist()

    # get accuracy
    accuracy.append(accuracy_score(y.tolist(), prediction))
print(f"Mean accuracy: {np.mean(accuracy)}")

Mean accuracy: 0.954173322683706


# With MNIST in-build data loader

### Train - batch size of 1

In [44]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import MNIST

# get as dataloaders
train_data = MNIST(root='mnist_data/train', train=True, download=True, transform=transforms.ToTensor())
test_data = MNIST(root='mnist_data/test', train=False, download=True, transform=transforms.ToTensor())

train_dataloader = DataLoader(dataset=train_data, batch_size=1)
test_dataloader = DataLoader(dataset=test_data, batch_size=1)

In [45]:
from time import time
import numpy as np

# define the model
clf = CNNClassifier().to(device)

# loss and optimization
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=clf.parameters(), lr=0.01)

start = time()
losses = []

# train loop
for epoch in range(5):
  # train
  clf.train()
  # iterate over the dataset
  for X, y in train_dataloader:
    # prepare the data
    X = X.to(device)
    y = y.to(device)

    # predict
    y_logit = clf(X)

    # loss
    loss = loss_fn(y_logit, y)
    losses.append(loss.cpu().detach().numpy())

    # update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # track loss
  print(f"Epoch {epoch}: loss - {np.mean(losses)}")

print("Time",time()-start)

Epoch 0: loss - 2.3101086616516113
Epoch 1: loss - 2.3100993633270264
Epoch 2: loss - 2.3100967407226562
Epoch 3: loss - 2.3100950717926025
Epoch 4: loss - 2.310094118118286
Time 832.3194110393524


In [46]:
# evaluate - Batch size of 1 only
with torch.inference_mode():
  # predictions
  prediction = []

  # test
  clf.eval()
  for X, y in test_dataloader:
    # prepare the data
    X = X.to(device)
    y = y.to(device)

    # predict
    y_logit = clf(X)
    prediction.append(torch.argmax(y_logit, dim=1).item())

# get accuracy
acc = accuracy_score(test_data_y, prediction)
print(f"Mean accuracy: {acc}")

Mean accuracy: 0.0958


### Train - batch size of 32

In [43]:
train_dataloader = DataLoader(dataset=train_data, batch_size=32)
test_dataloader = DataLoader(dataset=test_data, batch_size=32)

In [25]:
from time import time

# define the model
clf = CNNClassifier().to(device)

# loss and optimization
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=clf.parameters(), lr=0.01)

start = time()
losses = []

# train loop
for epoch in range(5):
  # train
  clf.train()
  # iterate over the dataset
  for X, y in train_dataloader:
    # prepare the data
    X = X.to(device)#.to(torch.float32).unsqueeze(dim=1)
    y = y.to(device)#.to(torch.int64)

    # predict
    y_logit = clf(X)

    # loss
    loss = loss_fn(y_logit, y)
    losses.append(loss.cpu().detach().numpy())

    # update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # track loss
  print(f"Epoch {epoch}: loss - {np.mean(losses)}")

print("Time",time()-start)

Epoch 0: loss - 0.16718541085720062
Epoch 1: loss - 0.1303280144929886
Epoch 2: loss - 0.11642495542764664
Epoch 3: loss - 0.10782646387815475
Epoch 4: loss - 0.10267454385757446
Time 61.4142370223999


In [42]:
# evaluate - Batch size of 32
with torch.inference_mode():
  # accuracy
  accuracy = []

  # test
  clf.eval()
  for X, y in test_dataloader:
    # prepare the data
    X = X.to(device)
    y = y.to(device)

    # predict
    y_logit = clf(X)
    prediction = torch.argmax(y_logit, dim=1).tolist()

    # get accuracy
    accuracy.append(accuracy_score(y.tolist(), prediction))
print(f"Mean accuracy: {np.mean(accuracy)}")

Mean accuracy: 0.965055910543131


# Conclusion -> both the models with manual and in-build dataloaders are outputting the similar results. What is important is to use a batchsize > 1, otherwise the model is not learning.